## Navigators
Navigators provide the ability to drill into the result set providing a summary of the distribution.  They are particularly useful when you are interested in gathering the domain of values for a particular Property.  Navigators can be used against a specific View, used in conjunction with either a query, a filter or both.  Navigators can be simple or very powerful, but provide a very useful way to capture results in logical buckets.  

In [2]:
import refinitiv.dataplatform as rdp
import pandas as pd
import datetime as dt
import dateutil.relativedelta

rdp.open_desktop_session('Your API Key here')

In [3]:
pd.set_option('display.max_colwidth', 140)
rdp.__version__

'1.0.0a7.post1'

#### Example - Property values
The most basic usage of navigators is to determine the available values within a given Property.  For example, when searching for assets, we may want to narrow down the specific country an asset is traded in.  We can determine the available countries to work with.

In [3]:
# The following will list the countries assets are listed.  We can use this output to filter based on a specific country.
response = rdp.Search.search(
    top = 0,
    navigators = "RCSCountryLeaf(buckets:20)"   # Show the top 20
)
response.data.raw["Navigators"]["RCSCountryLeaf"]["Buckets"]

[{'Label': 'United States', 'Count': 11826864},
 {'Label': 'Eurobond Market', 'Count': 3617762},
 {'Label': 'Colombia', 'Count': 1264380},
 {'Label': 'China (Mainland)', 'Count': 1231247},
 {'Label': 'South Korea', 'Count': 533448},
 {'Label': 'Germany', 'Count': 469694},
 {'Label': 'Canada', 'Count': 374139},
 {'Label': 'France', 'Count': 221040},
 {'Label': 'India', 'Count': 186007},
 {'Label': 'Japan', 'Count': 157262},
 {'Label': 'Chile', 'Count': 151908},
 {'Label': 'Italy', 'Count': 135888},
 {'Label': 'Australia', 'Count': 86496},
 {'Label': 'Spain', 'Count': 58537},
 {'Label': 'Switzerland', 'Count': 58249},
 {'Label': 'Russia', 'Count': 56044},
 {'Label': 'Austria', 'Count': 55356},
 {'Label': 'Norway', 'Count': 52819},
 {'Label': 'United Kingdom', 'Count': 50463},
 {'Label': 'Malaysia', 'Count': 49591}]

#### Example - Vessels
If I want to perform a basic search for tankers and capture the Hull Types, I could start with the following basic query:

In [4]:
rdp.search(
    view = rdp.SearchViews.VesselPhysicalAssets,
    query = "tanker",
    select = "DocumentTitle, HullType",
    top = 10000
)

,DocumentTitle,HullType
0,"FPSO YUUM KAK NAAB, LNG Tanker, BW OFFSHORE SINGAPORE PTE LTD, Gulf of Mexico|East Mexico Tanker Zone|Mexico",Single Hull
1,"MOZAH, LNG Tanker, NAKILAT SHIPPING QATAR LTD, North Pacific Ocean (Japan)|Taiwan Tanker Zone",Single Hull
2,"UMM SLAL, Liquefied Natural Gas; LNG Tanker, NAKILAT SHIPPING QATAR LTD, Java Sea|Singapore Tanker Zone|Singapore Strait Waypoint",Single Hull
3,"BU SAMRA, LNG Tanker, STASCO SHIP MANAGEMENT, Gulf of Oman|Arabian Gulf Tanker Zone|Qatar",Double Hull
4,"AL MAYEDA, LNG Tanker, STASCO SHIP MANAGEMENT, Gulf of Oman|Arabian Gulf Tanker Zone",Double Hull
...,...,...
9995,"CHANG FENG YOU 11, Other Tanker, East China Sea|Korea Tanker Zone|China (Mainland)",<NA>
9996,"HU CHONG HUO 0201, Other Tanker, Shanghai|East China Sea|Shanghai Tanker Zone|China (Mainland)",<NA>
9997,"ZHONG HAI 16, Other Tanker, Shanghai|Shanghai Tanker Zone|Yangtze River|China (Mainland)",<NA>
9998,"OOSTZEE, Other Tanker, English Channel|UK Continent Tanker Zone|ARA Tanker Zone|Netherlands",<NA>


Here, we can see a list of the different tankers and their Hull Types. Within this collection, while I can clearly see there are 2 different Hull Types: "Single" and "Double", I'm also seeing duplicates.  Instead of going through the effort to process the above result to pull out the unique list of Hull Types, I will instead use a Navigator - which was designed to do this work for me.

Using the above query, I will apply a Navigator by selecting the 'HullType' Property.  What this does is instruct search to bucket all unique Hull Types and summarize the total found for each. To capture the results, I will need to utilize the Content Layer API call.

In [5]:
response=rdp.Search.search(
    view = rdp.SearchViews.VesselPhysicalAssets,
    query = "tanker",
    top = 0,       # I'm not interested in the hits, only details within the navigator response section
    navigators = "HullType"
)
response.data.raw["Navigators"]["HullType"]["Buckets"]

[{'Label': 'Double Hull', 'Count': 16883},
 {'Label': 'Single Hull', 'Count': 5131}]

#### Example - Listing of exchanges per asset category
In the following example, I'm going to list the top 3 asset types in Canada and for each, display the top 2 exchanges trading on those assets.

In [6]:
response=rdp.Search.search(
    view = rdp.SearchViews.EquityQuotes,
    filter = "AssetState eq 'AC' and RCSExchangeCountryLeaf eq 'Canada'",    # Filter on all active (AC) assets within Canada
    top = 0,
    navigators = "RCSAssetCategoryLeaf(buckets:3, sub:ExchangeName(buckets:2))"  #  For the top 3 categories, show the top 2 exchanges
)

In [7]:
# Pretty display of the listing
rank = 1
for asset in response.data.raw["Navigators"]["RCSAssetCategoryLeaf"]["Buckets"]:
    print(f'{rank}. {asset["Label"]}')
    exchangeRank = 1
    for exchange in asset["ExchangeName"]["Buckets"]:
        print(f'\t{exchangeRank}. {exchange["Label"]} ({exchange["Count"]})')
        exchangeRank += 1
    rank += 1

1. Equity Cash Option
	1. Montreal Options Exchange (43216)
	2. Montreal Exchange (26)
2. Ordinary Share
	1. Aequitas NEO-L (3251)
	2. Aequitas NEO-N (3250)
3. Equity Future
	1. Montreal Exchange (7891)
	2. Montreal Options Exchange (109)


#### Example - Outstanding Government debt in the last 3 months.
In this example, I will be using some nested expressions to summarize categories within the buckets I collect.  Specifically, I'm looking for government debt ('govt') issued any time after 3-months ago using the following filter:
- "DBType eq 'govt' and IssueDate gt today-3months" (DBType identifies government debt)
    
Based on the result set, I want categorize or create buckets (top 5) based on the domicile of the issuer.  For each domicile, sum the outstanding debt and also include the maximum coupon rate for each one.
- RCSDomicileLeaf(buckets:5, desc:sum_FaceOutstandingUSD, calc:max_CouponRate)

In [6]:
# Get date N months ago
months = 3
date = dt.datetime.today() - dateutil.relativedelta.relativedelta(months=months)

In [9]:
# Retrieve the top 5 outstanding government debt in the last 3 months
response = rdp.Search.search(
    view = rdp.SearchViews.GovCorpInstruments,
    filter = "DbType eq 'govt' and IssueDate gt " + date.strftime("%Y-%m-%d"),
    top = 0,
    navigators = "RCSDomicileLeaf(buckets:5, desc:sum_FaceOutstandingUSD, calc:max_CouponRate)"
)
response.data.raw["Navigators"]["RCSDomicileLeaf"]["Buckets"]

[{'Label': 'United States',
  'Count': 69,
  'max_CouponRate': 1.625,
  'sum_FaceOutstandingUSD': 2311296405500.0},
 {'Label': 'Japan',
  'Count': 56,
  'max_CouponRate': 0.6,
  'sum_FaceOutstandingUSD': 1579333218038.0},
 {'Label': 'China (Mainland)',
  'Count': 46,
  'max_CouponRate': 3.97,
  'sum_FaceOutstandingUSD': 255746353840.0},
 {'Label': 'Hong Kong',
  'Count': 37,
  'max_CouponRate': 3.5,
  'sum_FaceOutstandingUSD': 138684763938.0},
 {'Label': 'Italy',
  'Count': 18,
  'max_CouponRate': 1.8,
  'sum_FaceOutstandingUSD': 123828421726.0}]